In [4]:
!pip install transformers datasets evaluate sacrebleu tf-keras ipywidgets sentencepiece sacremoses "accelerate>=0.26.0"

Defaulting to user installation because normal site-packages is not writeable


In [5]:
# import libraries
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)

2025-02-28 19:39:03.747260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740771543.830810    2092 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740771543.854566    2092 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# load dataset
dataset = load_dataset("qanastek/ELRC-Medical-V2", "en-fr")
ds = dataset["train"].train_test_split(test_size=0.2, seed=42) if "train" in dataset else dataset.train_test_split(test_size=0.2, seed=42)

README.md:   0%|          | 0.00/9.04k [00:00<?, ?B/s]

ELRC-Medical-V2.py:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13149 [00:00<?, ? examples/s]

In [7]:
# load pretrained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
def tokenize_fn(examples):
    en_texts = [ex["en"] for ex in examples["translation"]]
    fr_texts = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(en_texts, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(fr_texts, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_ds = ds.map(tokenize_fn, batched=True)

Map:   0%|          | 0/10519 [00:00<?, ? examples/s]

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2630 [00:00<?, ? examples/s]

In [10]:
# data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
# training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs"
)

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
# load bleu metric using evaluate
metric = evaluate.load("sacrebleu")

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[lbl.strip()] for lbl in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [17]:
# setup trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_2092/2610996801.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [18]:
# fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,0.635400,0.549707,36.972185
2,0.531700,0.535434,40.855252
3,0.478700,0.535532,45.469537
4,0.385800,0.535864,43.708938
5,0.349600,0.536489,42.591798
6,0.332700,0.542834,40.530393
7,0.287700,0.549456,42.164647
8,0.273800,0.552719,42.951368
9,0.263300,0.556894,43.452971
10,0.249000,0.558315,43.135228


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=6580, training_loss=0.3662485612562, metrics={'train_runtime': 4133.0447, 'train_samples_per_second': 25.451, 'train_steps_per_second': 1.592, 'total_flos': 2004377214320640.0, 'train_loss': 0.3662485612562, 'epoch': 10.0})

In [19]:
# evaluate the model on the test set
eval_results = trainer.evaluate(tokenized_ds["test"])
print("evaluation metrics:", eval_results)

evaluation metrics: {'eval_loss': 0.5583153963088989, 'eval_bleu': 43.1352278317791, 'eval_runtime': 345.3874, 'eval_samples_per_second': 7.615, 'eval_steps_per_second': 0.478, 'epoch': 10.0}


In [ ]:
# get a few original examples from the test set (for english and reference french)
sample_original = ds["test"].select(range(5))

# get predictions from the fine-tuned (sft) model
ft_predictions = trainer.predict(sample_test)
ft_decoded_preds = tokenizer.batch_decode(ft_predictions.predictions, skip_special_tokens=True)


# load the base model and set up a trainer for it with an eval_dataset
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_trainer = Seq2SeqTrainer(
    model=base_model,
    args=training_args,  # reuse same args
    eval_dataset=sample_test,  # add this line
    tokenizer=tokenizer,
    data_collator=data_collator
)
base_predictions = base_trainer.predict(sample_test)
base_decoded_preds = tokenizer.batch_decode(base_predictions.predictions, skip_special_tokens=True)


# print comparisons: original english, fine-tuned pred, base model pred, and reference french
for i, example in enumerate(sample_original):
    english_text = example["translation"]["en"]
    ref_text = example["translation"]["fr"]
    print(f"example {i}:")
    print("english      :", english_text)
    print("sft prediction:", ft_decoded_preds[i])
    print("base prediction:", base_decoded_preds[i])
    print("reference    :", ref_text)
    print("-" * 50)